In [1]:
!sudo apt install tesseract-ocr
!pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,850 kB of archives.
After this operation, 16.3 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1 [1,598 kB]
Get:2 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu focal/universe amd64 tesseract-ocr amd64 4.1.1-2build2 [262 kB]
Fetched 4,850 kB in 1s (3,506 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/Fro

In [2]:
!pip install easyocr
!pip install passporteye

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.9/813.9 kB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.0/146.0 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.7.0.72
    Uninstalling opencv-python-headless-4.7.0.72:
      Successfully uninstalled opencv-python-headless-4.7.0.72
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 38.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 96.7 MB/s eta 0:00:00
  Created wheel

<h3 style='font-weight: bold'>Import necesary packages</h3>

In [3]:
import os
import string as st
from dateutil import parser
import matplotlib.image as mpimg
import cv2
from passporteye import read_mrz
import json
import easyocr
import warnings
warnings.filterwarnings('ignore')

<h3 style='font-weight: bold'>Load easyOCR engine</h3>

In [4]:
# lOAD OCR ENGINE (easyOCR)
reader=easyocr.Reader(lang_list=['en'], gpu=True)  # Enable gpu if available

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

<h3 style='font-weight: bold'>Load country codes from <strong>country_codes.json</strong> file</h3>

In [8]:
!unrar x pass-project.rar


UNRAR 5.61 beta 1 freeware      Copyright (c) 1993-2018 Alexander Roshal


Extracting from pass-project.rar

Creating    pass-project                                              OK
Extracting  pass-project/a, b.png                                          0%  OK 
Extracting  pass-project/country_codes.json                                0%  OK 
Extracting  pass-project/dlib_face_recognition_resnet_model_v1.dat         4%  9% 13% 17% 22%  OK 
Extracting  pass-project/notebook.ipynb                                   22%  OK 
Extracting  pass-project/passport_1.png                                   22%  OK 
Extracting  pass-project/passport_temgoua.png                             23%  OK 
Extracting  pass-project/shape_predictor_68_face_landmarks.dat            27% 32% 36% 41% 45% 50% 54% 59% 63% 68% 72% 77% 81% 86% 90% 95% 99%  OK 
All OK


In [9]:
with open('/content/pass-project/country_codes.json') as f:
    country_codes = json.load(f)

<h3 style='font-weight: bold'>Define functions</h3>

In [19]:
!pip install face_recognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 11.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566173 sha256=3fb3b61ca935e4f63203f1df0ca68c71a1a66918f22002a9d6d7498e3dddd6c2
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [38]:
def match_images():
  import cv2
  import face_recognition

  # Load face and passport images
  image = input("Please Enter your image: ")
  passport = input("Please Enter your passport: ")
  face_img = face_recognition.load_image_file(image)
  passport_img = face_recognition.load_image_file(passport)

  # Find face encodings in both images
  face_encodings = face_recognition.face_encodings(face_img)
  passport_encodings = face_recognition.face_encodings(passport_img)

  if not face_encodings or not passport_encodings:
      print("Error: Could not detect a face in one or both of the images")
      exit()
  # Check face and passport image quality
  if face_img is None or passport_img is None:
      print('Error: Could not read input images')
      exit()
  elif face_img.shape[0] < 100 or face_img.shape[1] < 100:
      print('Error: Face image is too small')
      exit()
  elif passport_img.shape[0] < 300 or passport_img.shape[1] < 300:
      print('Error: Passport image is too small')
      exit()

  # Compare face encodings to check if they match
  matches = face_recognition.compare_faces(face_encodings, passport_encodings[0], tolerance=0.6)

  # Output result
  if True in matches:
      print("Face and passport belong to the same person")
  else:
      print("Face and passport belong to different persons")
  return passport


In [25]:
match_images()

Please Enter your image: /content/obama.jpg
Please Enter your passport: /content/pass-project/passport_temgoua.png
Face and passport belong to different persons


In [30]:
def parse_date(string, iob=True):
    date = parser.parse(string, yearfirst=True).date() 
    return date.strftime('%d/%m/%Y')

def clean(string):
    return ''.join(i for i in string if i.isalnum()).upper()

def get_country_name(country_code):
    country_name = ''
    for country in country_codes:
        if country['alpha-3'] == country_code:
            country_name = country['name']
            return country_name.upper()
    return country_code

def get_sex(code):
    if code in ['M', 'm', 'F', 'f']:
        sex = code.upper() 
    elif code == '0':
        sex = 'M'
    else:
        sex = 'F'
    return sex

def print_data(data):
    for key in data.keys():
        info = key.replace('_', ' ').capitalize()
        print(f'{info}\t:\t{data[key]}')
    return
  
def get_data(img_name):
    """ Extract personal info from img_name

    Args:
        img_name (str or fp): name or path of the passport image

    Returns:
        dict: dictionary of extracted data with keys and corresponding values as follows:
                surname         : surname
                name            : name
                sex             : sex
                date_of_birth   : date of birth
                nationality     : nationality
                passport_type   : passport type
                issuing_country : issuing country
                expiration_date : expiration date
                personal_number : personal number           
    """
    
    user_info = {}    
    new_im_path = 'tmp.png'
    im_path = img_name
    # Crop image to Machine Readable Zone(MRZ)
    mrz = read_mrz(im_path, save_roi=True)

    if mrz:
        mpimg.imsave(new_im_path, mrz.aux['roi'], cmap='gray')
    
        img = cv2.imread(new_im_path)
        img = cv2.resize(img, (1110, 140))
        
        allowlist = st.ascii_letters+st.digits+'< '
        code = reader.readtext(img, paragraph=False, detail=0, allowlist=allowlist)
        a, b = code[0].upper(), code[1].upper()
        
        if len(a) < 44:
            a = a + '<'*(44 - len(a))
        if len(b) < 44:
                b = b + '<'*(44 - len(b))
                
        surname_names = a[5:44].split('<<', 1)
        if len(surname_names) < 2:
            surname_names += ['']
        surname, names = surname_names
        
        user_info['name'] = names.replace('<', ' ').strip().upper()
        user_info['surname'] = surname.replace('<', ' ').strip().upper()
        user_info['sex'] = get_sex(clean(b[20]))
        user_info['date_of_birth'] = parse_date(b[13:19])
        user_info['nationality'] = get_country_name(clean(b[10:13]))
        user_info['passport_type'] = clean(a[0:2])
        user_info['passport_number']  = clean(b[0:9])
        user_info['issuing_country'] = get_country_name(clean(a[2:5]))
        user_info['expiration_date'] = parse_date(b[21:27])
        user_info['personal_number'] = clean(b[28:42])
        
    else:
        return print(f'Machine cannot read image {img_name}.')
    
    os.remove(new_im_path)
    
    return user_info

<h3 style='font-weight: bold'>Examples</h3>

In [ ]:
img_name = 'passport_temgoua.png'
data = get_data(img_name)
print_data(data)

Name	:	GILBERT
Surname	:	TEMGOUA DONKOO
Sex	:	M
Date of birth	:	09/04/1994
Nationality	:	CAMEROON
Passport type	:	PO
Passport number	:	1225801
Issuing country	:	CAR
Expiration date	:	06/05/2025
Personal number	:	M0966473


In [ ]:
img_name = '/content/passport_1.png'
data1 = get_data(img_name)
print_data(data1)

Name	:	MOHAMMED MUDASEER
Surname	:	
Sex	:	M
Date of birth	:	04/04/1982
Nationality	:	INDIA
Passport type	:	P
Passport number	:	H2908283
Issuing country	:	INDIA
Expiration date	:	09/02/2019
Personal number	:	


In [47]:
def passport_info():
  image_name = match_images()
  data = get_data(image_name)
  print_data(data)

In [48]:
passport_info()

Please Enter your image: /content/obama.jpg
Please Enter your passport: /content/pass-project/passport_temgoua.png
Face and passport belong to different persons
Name	:	GILBERT
Surname	:	TEMGOUA DONKOO
Sex	:	M
Date of birth	:	09/04/1994
Nationality	:	CAMEROON
Passport type	:	PO
Passport number	:	1225801
Issuing country	:	CAR
Expiration date	:	06/05/2025
Personal number	:	M0966473
